In [1]:
import boto3
from pprint import pprint
from datetime import datetime

In [2]:
session = boto3.Session(profile_name="default")
ddb = session.resource("dynamodb")

In [3]:
# Let's define table name
# REPLACE THE VALUE WITH YOUR OWN TABLE NAME!
table_name = "Cov19VaccinationDb-Cov19VaccinationTable07030B40-13FJE38K7H2H0"
table = ddb.Table(table_name)

## LEVEL 1: Basic Read and Write

In [4]:
# Put a sample data
resp = table.put_item(
    Item={
        "username": "sazae_huguta",
        "first_name": "Sazae",
        "last_name": "Huguta",
        "age": 24,
        "prefecture": "Tokyo",
        "dose": 1,
        "status": "reserved",
        "date": datetime(2021,7,20,hour=10,minute=0).isoformat(timespec="seconds"),
    }
)

In [5]:
# Get an item by id
resp = table.get_item(
    Key={"username": "sazae_huguta", "dose": 1}
)
pprint(resp["Item"])

{'age': Decimal('24'),
 'date': '2021-07-20T10:00:00',
 'dose': Decimal('1'),
 'first_name': 'Sazae',
 'last_name': 'Huguta',
 'prefecture': 'Tokyo',
 'status': 'reserved',
 'username': 'sazae_huguta'}


In [6]:
# Update an item by id
resp = table.update_item(
    Key={"username": "sazae_huguta", "dose": 1},
    UpdateExpression="SET prefecture = :val1",
    ExpressionAttributeValues={
        ":val1": "Aomori",
    }
)

In [7]:
# Check that the item was updated
resp = table.get_item(
    Key={"username": "sazae_huguta", "dose": 1}
)
pprint(resp["Item"]["prefecture"])

'Aomori'


In [8]:
# Update an item by id
resp = table.update_item(
    Key={"username": "sazae_huguta", "dose": 1},
    UpdateExpression="SET #at1 = :val1",
    ExpressionAttributeNames={
        '#at1': 'status'
    },
    ExpressionAttributeValues={
        ":val1": "completed",
    }
)

In [9]:
# Check that the item was updated
resp = table.get_item(
    Key={"username": "sazae_huguta", "dose": 1}
)
pprint(resp["Item"]["status"])

'completed'


In [10]:
# Delete an item
resp = table.delete_item(
    Key={"username": "sazae_huguta", "dose": 1},
)

In [13]:
# Check that the item was updated
resp = table.get_item(
    Key={"username": "sazae_huguta", "dose": 1},
)
if resp.get("Item"):
    print(resp.get("Item"))
else:
    print("The item with the given ID was not found!")

The item with the given ID was not found!


## LEVEL 2: Query and Scan

In [14]:
import json

In [15]:
with open("data.json", "r") as f:
    data = json.load(f)

In [16]:
# Batch-write data
with table.batch_writer() as batch:
    for d in data:
        batch.put_item(Item=d)

In [17]:
from boto3.dynamodb.conditions import Key, Attr

In [18]:
# query by name
resp = table.query(
    KeyConditionExpression=Key('username').eq('namihei_isono')
)
pprint(resp.get("Items"))

[{'age': Decimal('54'),
  'date': '2021-07-25T10:00:00',
  'dose': Decimal('1'),
  'first_name': 'Namihei',
  'last_name': 'Isono',
  'prefecture': 'Chiba',
  'status': 'completed',
  'username': 'namihei_isono'},
 {'age': Decimal('54'),
  'date': '2021-08-20T10:00:00',
  'dose': Decimal('2'),
  'first_name': 'Namihei',
  'last_name': 'Isono',
  'prefecture': 'Chiba',
  'status': 'completed',
  'username': 'namihei_isono'}]


In [19]:
# query by name and dose
resp = table.query(
    KeyConditionExpression=Key("username").eq("namihei_isono") & Key('dose').eq(1)
)
pprint(resp.get("Items"))

[{'age': Decimal('54'),
  'date': '2021-07-25T10:00:00',
  'dose': Decimal('1'),
  'first_name': 'Namihei',
  'last_name': 'Isono',
  'prefecture': 'Chiba',
  'status': 'completed',
  'username': 'namihei_isono'}]


In [20]:
# query by age
resp = table.query(
    IndexName="ItemsByAge",
    KeyConditionExpression=Key('age').eq(11),
)
pprint(resp.get("Items"))

[{'age': Decimal('11'),
  'date': '2021-07-20T10:00:00',
  'dose': Decimal('1'),
  'first_name': 'Katsuo',
  'last_name': 'Isono',
  'prefecture': 'Gunma',
  'status': 'reserved',
  'username': 'katsuo_isono'}]


In [21]:
# query by prefecture
resp = table.query(
    IndexName="ItemsByPrefecture",
    KeyConditionExpression=Key('prefecture').eq("Tokyo"),
)
pprint(resp.get("Items"))

[{'age': Decimal('3'),
  'date': '2021-07-20T10:00:00',
  'dose': Decimal('1'),
  'first_name': 'Tarao',
  'last_name': 'Huguta',
  'prefecture': 'Tokyo',
  'status': 'reserved',
  'username': 'tarao_huguta'},
 {'age': Decimal('28'),
  'date': '2021-07-20T10:00:00',
  'dose': Decimal('1'),
  'first_name': 'Huguta',
  'last_name': 'Masuo',
  'prefecture': 'Tokyo',
  'status': 'reserved',
  'username': 'masuo_huguta'}]


In [22]:
# Scan - simple
resp = table.scan()
items = resp.get("Items")
print("Number of items:", len(items))

Number of items: 8


In [23]:
# scan - all data
resp = table.scan()
items = resp.get("Items")
# query all items!
while resp.get("LastEvaluatedKey"):
    resp = table.scan(ExclusiveStartKey=r["LastEvaluatedKey"])
    items.extend(resp["Items"])
print("Number of items", len(items))

Number of items 8


In [24]:
# scan with filtering
resp = table.scan(
    FilterExpression=Attr('age').lt(27)
)
pprint(resp.get("Items"))

[{'age': Decimal('9'),
  'date': '2021-07-20T10:00:00',
  'dose': Decimal('1'),
  'first_name': 'Wakame',
  'last_name': 'Isono',
  'prefecture': 'Saitama',
  'status': 'reserved',
  'username': 'wakame_isono'},
 {'age': Decimal('3'),
  'date': '2021-07-20T10:00:00',
  'dose': Decimal('1'),
  'first_name': 'Tarao',
  'last_name': 'Huguta',
  'prefecture': 'Tokyo',
  'status': 'reserved',
  'username': 'tarao_huguta'},
 {'age': Decimal('11'),
  'date': '2021-07-20T10:00:00',
  'dose': Decimal('1'),
  'first_name': 'Katsuo',
  'last_name': 'Isono',
  'prefecture': 'Gunma',
  'status': 'reserved',
  'username': 'katsuo_isono'}]


In [25]:
# scan by date
resp = table.scan(
    FilterExpression=Attr('date').begins_with("2021-07-25"),
)
pprint(resp.get("Items"))

[{'age': Decimal('54'),
  'date': '2021-07-25T10:00:00',
  'dose': Decimal('1'),
  'first_name': 'Namihei',
  'last_name': 'Isono',
  'prefecture': 'Chiba',
  'status': 'completed',
  'username': 'namihei_isono'}]


In [26]:
# get a subset of the attributes
resp = table.scan(
    ProjectionExpression="first_name, prefecture"
)
pprint(resp.get("Items"))

[{'first_name': 'Wakame', 'prefecture': 'Saitama'},
 {'first_name': 'Namihei', 'prefecture': 'Chiba'},
 {'first_name': 'Namihei', 'prefecture': 'Chiba'},
 {'first_name': 'Huguta', 'prefecture': 'Tokyo'},
 {'first_name': 'Hune', 'prefecture': 'Chiba'},
 {'first_name': 'Hune', 'prefecture': 'Chiba'},
 {'first_name': 'Tarao', 'prefecture': 'Tokyo'},
 {'first_name': 'Katsuo', 'prefecture': 'Gunma'}]


## LEVEL 3: Backing up a Table

In [27]:
client = session.client("dynamodb")

In [28]:
resp = client.create_backup(
    TableName=table_name,
    BackupName=table_name + "-Backup"
)
backup_arn = resp["BackupDetails"]["BackupArn"]
print(backup_arn)

arn:aws:dynamodb:ap-northeast-1:606887060834:table/Cov19VaccinationDb-Cov19VaccinationTable07030B40-13FJE38K7H2H0/backup/01626402955980-109ca39b


In [29]:
resp = client.describe_backup(BackupArn=backup_arn)
pprint(resp["BackupDescription"]["BackupDetails"])

{'BackupArn': 'arn:aws:dynamodb:ap-northeast-1:606887060834:table/Cov19VaccinationDb-Cov19VaccinationTable07030B40-13FJE38K7H2H0/backup/01626402955980-109ca39b',
 'BackupCreationDateTime': datetime.datetime(2021, 7, 16, 11, 35, 55, 980000, tzinfo=tzlocal()),
 'BackupName': 'Cov19VaccinationDb-Cov19VaccinationTable07030B40-13FJE38K7H2H0-Backup',
 'BackupSizeBytes': 0,
 'BackupStatus': 'AVAILABLE',
 'BackupType': 'USER'}


In [30]:
# modify a table
resp = table.update_item(
    Key={"username": "tarao_huguta", "dose": 1},
    UpdateExpression="SET age = :val1",
    ExpressionAttributeValues={
        ":val1": 4
    }
)

# and confirm that the item was updated
resp = table.get_item(
    Key={"username": "tarao_huguta", "dose": 1},
)
pprint(resp["Item"]["age"])

Decimal('4')


In [31]:
# restore from the backup
restored_table_name = table_name + "-restored"
resp = client.restore_table_from_backup(
    TargetTableName=restored_table_name,
    BackupArn=backup_arn,
)

In [32]:
# Check the state of the restored table
resp = client.describe_table(TableName=table_name + "-restored")
pprint(resp["Table"]["TableStatus"])

'CREATING'


In [33]:
# Check the state of the restored table
resp = client.describe_table(TableName=table_name + "-restored")
pprint(resp["Table"]["TableStatus"])

'ACTIVE'


In [34]:
restored_table = ddb.Table(restored_table_name)
resp = restored_table.get_item(
    Key={"username": "tarao_huguta", "dose": 1},
)
pprint(resp["Item"]["age"])

Decimal('3')


In [35]:
# Delete restored table
resp = restored_table.delete()

In [36]:
# Delete backup
resp = client.delete_backup(BackupArn=backup_arn)